In [1]:
import os
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '/home/djl34/lab_pd/kl/git/KL/scripts')
import post_analysis

pd.set_option('display.max_columns', None)

pd_data_dir = "/home/djl34/lab_pd/data"
aso_data_dir = "/home/djl34/lab_pd/aso/data"
KL_data_dir = "/home/djl34/lab_pd/kl/data"
scratch_dir = "/n/scratch/users/d/djl34"

all_chrom_set = [str(x) for x in range(1, 23)]
chrom_set = all_chrom_set

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
client = Client()

In [3]:
filename = os.path.join(KL_data_dir, "results/footprints/mu_sum/{chrom}.tsv")
filename = filename.replace("{chrom}", "*")

In [4]:
ddf = dd.read_csv(filename, sep = "\t", dtype={'Spliceai_info': 'object'})

In [5]:
df = ddf.compute()

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


In [6]:
filename = os.path.join(KL_data_dir, "results/footprints/phyloP/{chrom}.tsv")
filename = filename.replace("{chrom}", "*")
ddf = dd.read_csv(filename, sep = "\t")
df_phyloP = ddf.compute()

df = df.merge(df_phyloP, on = "footprint_identifier", how = "left")

In [42]:
df.head()

,footprint_identifier,mu,expected_AC,AC_var,AC_total,denovo_an_case,denovo_an_control,AN_mean,sites,phyloP,phyloP_pos,phyloP_max,phyloP_median
0,1.102814.3,4.278,564.329566,5.723804e+06,21.0,0.0,0.0,550917.515152,33,-0.672000,0.102909,0.568,-0.3530
1,1.102814.44,11.287,1496.473430,1.509851e+07,432.0,0.0,0.0,549097.975309,81,-0.962383,0.155370,0.545,-0.4060
2,1.102814.47,16.900,2219.699413,2.249524e+07,134.0,0.0,0.0,550766.588235,136,-0.580324,0.056081,0.545,0.0520
3,1.102821.4,6.963,902.101016,9.095739e+06,473.0,0.0,0.0,552212.023810,84,-0.301964,0.278643,1.083,-0.0540
4,1.1029632.1,6.412,833.497037,8.489607e+06,197.0,0.0,0.0,552300.055556,90,0.230933,0.340833,0.572,0.4375


In [43]:
## calculate z_score
df["Z_score"] = (df["AC_total"] - df["expected_AC"])/np.sqrt(df["AC_var"])

In [44]:
print(df["Z_score"].min())
print(df["Z_score"].max())
print(df["Z_score"].mean())

-2.7309230501550448
57.304889797978035
-0.03433378906555952


In [45]:
len(df[df["Z_score"] < -1])/len(df)

3.032569305213916e-05

In [46]:
len(df[df["Z_score"] < -1])

124

In [47]:
df.head()

,footprint_identifier,mu,expected_AC,AC_var,AC_total,denovo_an_case,denovo_an_control,AN_mean,sites,phyloP,phyloP_pos,phyloP_max,phyloP_median,Z_score
0,1.102814.3,4.278,564.329566,5.723804e+06,21.0,0.0,0.0,550917.515152,33,-0.672000,0.102909,0.568,-0.3530,-0.227102
1,1.102814.44,11.287,1496.473430,1.509851e+07,432.0,0.0,0.0,549097.975309,81,-0.962383,0.155370,0.545,-0.4060,-0.273948
2,1.102814.47,16.900,2219.699413,2.249524e+07,134.0,0.0,0.0,550766.588235,136,-0.580324,0.056081,0.545,0.0520,-0.439751
3,1.102821.4,6.963,902.101016,9.095739e+06,473.0,0.0,0.0,552212.023810,84,-0.301964,0.278643,1.083,-0.0540,-0.142279
4,1.1029632.1,6.412,833.497037,8.489607e+06,197.0,0.0,0.0,552300.055556,90,0.230933,0.340833,0.572,0.4375,-0.218450


In [48]:
per_generation_factor = 1.015 * 10 **-7

df["rate_pergen"] = df["mu"] * per_generation_factor * 1902

In [80]:
df_footprint_mean = pd.DataFrame(df[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean(), columns= ["all"])

## do it for a cutoff

### Z_score < -1

In [81]:
df_zscore_1 = df[df["Z_score"] < -1]

df_zscore_1["denovo_an_case"].sum()/df_zscore_1["rate_pergen"].sum()

df_zscore_1["denovo_an_control"].sum()/df_zscore_1["rate_pergen"].sum()

df_footprint_mean["Zscore < -1"] = df_zscore_1[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

In [57]:
df_footprint_mean

,all,Zscore < 1
phyloP,0.500888,2.104707
phyloP_pos,0.839108,2.377669
phyloP_max,2.503774,5.509202
phyloP_median,0.547084,2.128012


### Z_score < 0.5

In [85]:
df_zscore = df[df["Z_score"] < -0.5]

print(len(df_zscore))

print(df_zscore["denovo_an_case"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_control"].sum()/df_zscore["rate_pergen"].sum())

df_footprint_mean["Zscore < -0.5"] = df_zscore[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

5567
1.0454997190594053
0.6571712519801975


In [73]:
df_zscore["denovo_an_case"].sum()

35.0

In [74]:
df_zscore["denovo_an_control"].sum()

22.0

## other cutoffs

In [87]:
df_zscore = df[df["Z_score"] < - 0.7]

print(len(df_zscore))

print(df_zscore["denovo_an_case"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_case"].sum())
print(df_zscore["denovo_an_control"].sum()/df_zscore["rate_pergen"].sum())

df_footprint_mean["Zscore < -0.7"] = df_zscore[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

435
0.7295795988401905
5.0
0.0


In [82]:
df_zscore = df[df["Z_score"] < - 0.8]

print(len(df_zscore))

print(df_zscore["denovo_an_case"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_control"].sum()/df_zscore["rate_pergen"].sum())

df_footprint_mean["Zscore < -0.8"] = df_zscore[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

250
0.0
0.0


In [88]:
df_zscore = df[df["Z_score"] < - 0.6]

print(len(df_zscore))

print(df_zscore["denovo_an_case"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_case"].sum())
print(df_zscore["denovo_an_control"].sum()/df_zscore["rate_pergen"].sum())
print(df_zscore["denovo_an_control"].sum())


df_footprint_mean["Zscore < -0.6"] = df_zscore[["phyloP", "phyloP_pos", "phyloP_max", "phyloP_median"]].mean()

1156
0.8410047203935402
10.0
0.3364018881574161
4.0


In [86]:
df_footprint_mean

,all,Zscore < -1,Zscore < -0.8,Zscore < -0.7,Zscore < -0.6,Zscore < -0.5
phyloP,0.500888,2.104707,1.978092,1.805940,1.346052,0.988912
phyloP_pos,0.839108,2.377669,2.247573,2.098740,1.687538,1.377093
phyloP_max,2.503774,5.509202,5.239268,5.156821,4.770325,4.366154
phyloP_median,0.547084,2.128012,2.016986,1.861807,1.417084,1.073150
